In [6]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [32]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = '6DTK9yDP6p0'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [33]:
topics = [{
	"start": 72,
	"finish": 288
},
{
	"start": 288,
	"finish": 329
},
{
	"start": 329,
	"finish": 410
},
{
	"start": 410,
	"finish": 513
},
{
	"start": 513,
	"finish": 604
},
{
	"start": 604,
	"finish": 702
},
{
	"start": 702,
	"finish": 955 
},
{
	"start": 955,
	"finish": 1139 
},
{
	"start": 1139,
	"finish": 1339
},
{
	"start": 1339,
	"finish": 1420
},
{
	"start": 1420,
	"finish": 1575
},
{
	"start": 1575,
	"finish": 1606
}]


In [ ]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text

def getTopic(topic, video_id):
    return {
        "name": "",
        "desctiption": "",
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start"],
            "end": topic["finish"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }
startNum = 16
startChar = 'b'

for topic in topics[1:]:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    print('"' + str(startNum) + startChar + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',') 
    startChar = chr(ord(startChar) + 1)
    print(resp.choices[0].message.content)
    print("====================================")

"16b": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "6DTK9yDP6p0",
            "start": 288,
            "end": 329
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
1. Topic Name: Prioritization in Startups: Time Management and Making Impactful Decisions 

2. Very very small description of the topic: This topic explains the importance of prioritization in managing time effectively in a startup and provides guidance on effectively aligning with co-founder's expectations.

3. Rewrite of the topic:

Hello folks! Today, we're going to delve into a vital aspect of running a startup, and that's **prioritization**. Now, when we talk about prioritizing, there are essentially two ways to look at it. 

First, there's how you divide your time between working on your startup and managing other areas of your life. Now, I can't tell you exactly how to handle thi